<a href="https://colab.research.google.com/github/eduardocorrearaujo/ModeloSIR/blob/master/SIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo Epidemiológico **SIR**
A pandemia do COVID-19, também chamado de "o novo coronavírus", colocou em evidência a importância da matemática em nosso cotidiano. Tenho certeza que nesse período você já ouviu falar sobre projeções do avanço da doença e pico da curva de infectados. Porém, você sabe como a matemática se relaciona com esses termos?

<p align="center">
<img alt="covid" width="25%" src="https://github.com/eduardocorrearaujo/ModeloSIR/blob/master/imcovid.png?raw=true">
</p>
<center>
<a href="https://br.freepik.com/fotos-vetores-gratis/medico">Médico vetor criado por freepik - br.freepik.com</a>
</center>

Para compreender esses termos e realizar essas análise podemos utilizar modelos matemáticos, que são, a grosso modo, uma simplificação da realidade. No contexto de epidemias podem ser aplicados modelos compartimentais, que dividem a população em "caixinhas" para assim, tentar entender o comportamento de uma doença no decorrer do tempo. Um dos modelos compartimentais mais simples é o modelo epidemiológico **SIR**, proposto em 1927 por Kermack e McKendrick.

Esse modelo divide a população em 3 grupos, **Suscetíveis (S)**, **Infectados (I)** e **Recuperados (R)**. Suscetíveis  representam a parcela da população que
        não foi infectada, os infectados representam as pessoas que estão infectadas com o vírus, transmitindo para outras pessoas e os recuperados são aqueles que
        já se curaram, ou seja, estavam infectados, mas agora estão curados. Veja o diagrama abaixo.
<p align="center">
<img alt="sir" width="40%" src="https://github.com/eduardocorrearaujo/ModeloSIR/blob/master/modeloSIR.png?raw=true">
</p>

No digrama temos duas letras $\beta$ e $\gamma$ elas representam a taxa em que os indivíduos se movem de um grupo para o outro. O
$\beta$ representa a **taxa de contágio**, ela vai ser a responsável por "mover" uma pessoa do grupo dos suscetíveis para o grupo dos infectados. Ela está relacionada
a probabilidade de uma transmissão ocorrer decorrente do contato entre um indivíduo infectado e um indivíduo suscetível.  O $\gamma$ representa a **taxa  de recuperação**, ela é responsável por "mover"
o indivíduo do grupo dos infectados para o grupo dos recuperados. O inverso do $\gamma$, ($1/\gamma$) é muito importante, ele representa o **período infeccioso médio**, ou seja ele
nos fornece uma estimativa para quanto tempo o indivíduo estará infectando outras pessoas.

Matematicamente, o modelo SIR é definido pelo seguinte sistema de equações diferenciais ordinárias:

\begin{cases}
S'(t) = -\dfrac{\beta S(t) I(t)}{N}\\
I'(t) = \dfrac{\beta S(t) I(t)}{N} -  \gamma I(t)\\
R'(t) = \gamma I(t)
\end{cases}
                
Sendo $\beta$ a taxa de contágio, $\gamma$ a taxa de recuperação e $N$ a população total da região que estamos analisando. 

Esse modelo nos fornece uma informação muito importante sobre o desenvolvimento de uma epidemia. Ele nos fornece uma estimativa  da **taxa básica de
reprodução da doença/vírus**, que no caso do COVID-19 seria uma média de quantas pessoas podem ser infectadas por um indivíduo infectado em contato com os suscetíveis. Essa taxa é dada por $R_0 = \dfrac{\beta}{\gamma}$,
sendo importante ressaltar que $\dfrac{1}{\gamma}$ é o perído infeccioso médio e $\beta$ a taxa de contágio.

Veja abaixo como podemos implementar esse modelo utilizando a linguagem de programação **python**. Nesse caso, como esse modelo não tem solução analítica conhecida vamos utilizar o python para gerar uma solução númerica utilizando o comando `odeint` e vamos plotar essa solução utilizando a biblioteca `plotly`.



In [0]:
#pacotes necessário para análise
from scipy.integrate import odeint
import numpy as np
import plotly.graph_objects as go


Define a função que irá fornecer a solução númerica do sistema de equações.

In [0]:
#Solução numérica do modelo SIR
def SIR(t,beta,gama,N, i0,r0): 
    def dotSIR(y,t,beta,gama,N): # função que define o sistema de EDO's
      S,I,R = y # # y é o vetor com as 3 equações S,I,R
      dydt = [-beta*S*I/N,beta*S*I/N - gama*I, gama*I ] #vetor em que cada elemento é uma das equações
      return dydt
    #S = Pessoas que suscetíveis, ou seja, que podem ser infectadas.
    #I = Pessoas que foram infectadas.
    #R = Pessoas que se recuperaram.
    #beta = Taxa de transmissão da doença.
    #gama = Taxa de recuperação.
    #N = População total.


    # Como se trata de um sistema de EDO's precisamos das condições iniciais para resolvê-lo.
    #Como esse modelo considera a populção constante vamos considerar s0 = N - i0 - r0
    # N = população total.
    #s0 = número inicial de pessoas Suscetíveis.
    #i0 = número inicial de pessoas Infectadas.
    #r0 = número inicial de pessoas Recuperadas.
    s0 = N - i0 - r0
    y0 = [s0,i0,r0] #vetor que armazena as condições iniciais do sistema

    sol = odeint(dotSIR, y0,t, args=(beta,gama,N)) 
    #O comando responsável por encontrar a solução numérica é o odeint
    # O comando retorna uma matriz com as colunas determinadas pelo número de equações, nesse caso 3 
    # e com o número de linhas referentes ao intervalo de tempo considerado.
    
    S = sol[:,0] 
    # a primeira coluna da matriz sol é referente aos Suscetíves, que é a primeira equação do sistema.
    I = sol[:,1] 
    # a segunda coluna da matriz é referente aos Infectados, que é a segunda equação do sistema.
    R = sol[:,2] 
    # a terceira coluna da matriz é referente aos Recuperados, que é a terceira equação do sistema.
    C = sol[:, 1] + sol[:,2] 
    #variável que armazena a soma dos Infectados + Recuperados com o tempo para podermos analisar o
    #valor acumulado de pessoas infectadas no decorrer da epidemia

    return S,I,R,C

Agora que definimos a função que irá calcular a solução númerica do sistema, podemos definir as condições iniciais do problema e usar a biblioteca **plotly** para visualisarmos o resultado.

In [0]:
t = np.arange(300) 
#gera um vetor (0,1,2,3..,299) que representa o valores de t usados para gerar a solução.

#parâmetros para gerar a solução:
beta = 0.18
gama = 0.047
N = 1000
i0 = 1
r0 = 0


solucao = SIR(t,beta,gama,N, i0,r0) 
#variável que armazenará S,I,R,C que são os valores retornados pela função SIR,
#como a função retorna um array com cada elemento sendo S,I,R,C para analisar cada solução separadamente  
#vamos definir as seguintes variáveis
S = solucao[0]  #Suscetíves
I = solucao[1]  #Infectados
R = solucao[2]  #Recuperados
C = solucao[3]  #Casos Acumulados

#cada uma das variáveis trace abaixo armazena o conjunto de pontos gerada pela função SIR que queremos plotar. 
#Veja que como queremos plotar 4 conjuntos de dados diferentes S,I,R,C armazenamos cada um deles em uma varíavel.
trace1 = go.Scatter(x=t, y=S,mode='lines',name='Suscetíveis',) 

trace2 = go.Scatter(x=t, y=I, mode='lines',name='Infectados',)

trace3 = go.Scatter(x=t, y=R, mode='lines', name='Recuperados',)

trace4 = go.Scatter(x=t, y=C,mode='lines',name='Casos acumulados',)

layout = go.Layout( title = 'Modelo SIR', #define o título do gráfico 
                  titlefont = {'family': 'Arial', 'size': 22,'color': '#7f7f7f'}, #define a fonte e cor e fonte do título
                  xaxis = {'title': 'Dias'}, #título do eixo x
                  yaxis = {'title': 'População'},  #título do eixo y
                       )

data=[trace1,trace2,trace3,trace4] #vetor que armazena os pontos que queremos plotar

fig = go.Figure(data, layout = layout) #função que gera a figura
       
fig.update_layout(
legend_orientation="h", #define a localização da legenda
legend=dict(
         y=-0.15,       #coloca um recuo na legenda para que ela fique mais longe do título do eixo x.
        ))

fig.show() #mostra a figura

R0 = beta/gama
#Caso deseje que o R0 seja printado adicione o seguinte comando:
#print('Para essas condições o R0 é:%f' %R0)


É importante ressaltar que um modelo é apenas uma aproximação da realidade. O modelo SIR é um dos mais básicos, por essa razão ele não é o modelo ideal para fazer projeções. Podemos torná-lo mais próximo da realidade adicionando mais compartimentos (caixinhas) como por exemplo, um compartimento para os mortos, os quarentenados e os expostos, aqueles que foram expostos ao vírus mais ainda não estão infectando outras pessoas. 

